In [ ]:
import re
from html.parser import HTMLParser
import pandas as pd
import emoji
import string
import fasttext
from nltk.corpus import stopwords
import spacy
from spacy.lang.ru import Russian
import concurrent.futures

contractions = {
    "и т.д.": " и так далее ",
    "и т.п.": " и тому подобное ",
    "ул.": " улица "
}

nlp = spacy.load('ru_core_news_lg')

In [ ]:
class MyHTMLParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.extracted_data = []

    def handle_data(self, data):
        self.extracted_data.append(data)

    def get_data(self):
        return ''.join(self.extracted_data)

In [ ]:
def remove_non_cyrillic(text):
    # Удаляем все, кроме русских букв
    return re.sub(r'[^а-яА-ЯёЁ]', ' ', text)

In [ ]:
def replace_r_with_rubles(text):
    # Замена "р" после числа
    text = re.sub(r'(\d)р\b', r'\1 рублей ', text)
    # Замена "р" перед числом
    text = re.sub(r'\bр(\d)', r' рублей \1', text)
    return text

In [ ]:
def replace_currency_symbols(text):
    # Словарь символов валют и их полных названий
    currency_symbols = {
        r'\$': ' долларов ',
        r'€': ' евро ',
        r'£': ' фунтов ',
        r'¥': ' йен ',
        r'₽': ' рублей '
    }
    
    # Перебираем словарь и заменяем каждый символ валюты в тексте
    for symbol, name in currency_symbols.items():
        text = re.sub(symbol, name, text)
    
    return text

In [ ]:
def remove_punctuation_list_comp(text):
    return re.sub(r'\s+', ' ', re.sub(r'[\s{}]+'.format(re.escape(string.punctuation)), ' ', text)).strip()

In [ ]:
def expand_contractions(text, contractions_dict):
    for key, value in contractions_dict.items():
        text = text.replace(key, value)
    return text

In [ ]:
def replace_units_with_full_names(text):
    # Словарь замен: сокращение единицы измерения и его полное словесное представление
    units = {
        'кг': 'килограмм',
        'г': 'грамм',
        'м': 'метр',
        'см': 'сантиметр',
        'мм': 'миллиметр',
        'л': 'литр',
        'мл': 'миллилитр',
        'ч': 'час',
        'мин': 'минута',
        'сек': 'секунда',
        'км': 'километр',
        'шт': 'штук'
    }
    
    for unit, full_name in units.items():
        text = text.replace(f' {unit} ', f' {full_name} ')
    return text

In [ ]:
def replace_hyphens(text):
    # Замена дефиса на минус в математических выражениях
    text = re.sub(r'(?<=\d)\-(?=\d)', ' минус ', text)  # между цифрами
    text = re.sub(r'(?<=\s)\-(?=\d)', ' минус ', text)  # перед числом после пробела
    
    # Замена дефиса на тире, учитывая контексты, где дефис не заменяется (например, частицы)
    # Различие между тире и дефисом в частицах (например, "что-то", "кто-либо")
    text = re.sub(r'\b(\w+)-(то|либо|нибудь|таки|ка)\b', r'\1-\2', text)

    # Общее правило замены дефиса на тире, если это не частицы
    text = re.sub(r'(?<!\w)(\w+)-(\w+)', r'\1 дефис \2', text)

    # Замена дефиса на тире в прочих случаях
    text = re.sub(r'(?<=\s)-(?=\s)', ' дефис ', text)  # между пробелами
    text = re.sub(r'(?<=\w)-(?=\s)', ' дефис ', text)  # после слова перед пробелом

    return text

In [ ]:
def remove_extra_spaces_regex(text):
    text = re.sub(r'\s+', ' ', text.strip())
    return text

In [ ]:
def replace_identifiers(text):
    # Регулярное выражение, которое ищет шестнадцатеричные хеши, UUIDs и другие типичные идентификаторы
    pattern = r'\b([a-f0-9]{32}|[a-f0-9]{40}|[a-f0-9]{64}|[a-f0-9-]{36}|[a-zA-Z0-9-]{7,})\b'
    
    # Заменяем найденные идентификаторы на слово "идентификатор"
    return re.sub(pattern, ' идентификатор ', text, flags=re.IGNORECASE)

In [ ]:
def replace_phone_numbers_and_digits(text):
    # Регулярное выражение для номеров телефонов
    phone_pattern = r'\+?\d[\d\-\(\)\.\s]{8,}\d'
    
    # Заменить номера телефонов на "номер телефона"
    text = re.sub(phone_pattern, ' номер телефона ', text)

    text = replace_identifiers(text)
    
    # Заменить оставшиеся числа на "число"
    text = re.sub(r'\b\d+\b', ' число ', text)
    
    return text

In [ ]:
def replace_custom_text_emojis(text):
    # Словарь текстовых эмодзи и их словесных описаний
    emoji_dict = {
        r':\)': ' улыбка ',
        r':\(': ' грустное лицо ',
        r':D': ' смех ',
        r';\)': ' подмигивание '
    }
    
    # Перебираем словарь и заменяем каждый эмодзи в тексте
    for emoji, description in emoji_dict.items():
        text = re.sub(emoji, description, text, flags=re.IGNORECASE)
    
    return text

In [ ]:
def replace_math_symbols_with_words(text):
    # Словарь замен: математический символ и его словесное представление
    math_symbols = {
        '+': ' плюс ',
        '*': ' умножить на ',
        '/': ' разделить на ',
        '=': ' равно ',
        '<': ' меньше ',
        '>': ' больше ',
        '≤': ' меньше или равно ',
        '≥': ' больше или равно '
    }
    
    # Перебираем словарь и заменяем каждый символ в тексте
    for symbol, word in math_symbols.items():
        text = text.replace(symbol, word)
    
    return text

In [ ]:
def replace_slang(text):
    # Словарь замен: сленг и его нормальное представление
    slang_dict = {
        r'\bтг\b': ' телефон ',
        r'\bспс\b': ' спасибо ',
        r'\bплиз\b': ' пожалуйста ',
        r'\bчел\b': ' человек ',
        r'\bкек\b': ' смешно ',
        r'\bлол\b': ' смешно '
    }
    
    # Перебираем словарь и заменяем каждый сленг в тексте
    for slang, normal in slang_dict.items():
        text = re.sub(slang, normal, text, flags=re.IGNORECASE)
    
    return text

In [ ]:


def preprocess_text(text):

    text = text.lower()

   # text = replace_hyphens(text)

    #print("hyphens ", text)

    text = replace_slang(text)

    #print("slang ", text)

    text = replace_custom_text_emojis(text)

    #print("emoji ", text)

    text = replace_r_with_rubles(text)

    #print("rubles ", text)

    text = replace_currency_symbols(text)

    #print("currency ", text)

    text = replace_phone_numbers_and_digits(text)

    #print("phone_numbers_and_digits ", text)

    # Перевод в нижний регистр

    
    # Раскрытие сокращений
    text = expand_contractions(text, contractions)

    """   url_pattern = re.compile(
    r'\b(?:https?|ftp|mailto|data|tel):\/\/'  # Расширенные схемы
    r'(?:(?:[a-z0-9-]+\.)+[a-z]{2,13})'  # Доменное имя
    r'(?:\/[\w\-\.~:+\/?#\[\]@!$&\'()*;,=]*)?'  # Путь
    r'(?:(?:\?[\w\-\.~:+\/?#\[\]@!$&\'()*;,=]*)?)'  # Параметры
    r'(?:(?:#[\w\-]*)?)\b',  # Якорь
    re.IGNORECASE)
    """


    url_pattern = re.compile(r'https?://(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,}')

    # Удаление ссылок
    text = re.sub(url_pattern, '', text)

    #print("links ", text)

    # переводим эмоджи в текст вида :машет_рукой: , заменяем символы : и _ на пробелы
    text = emoji.demojize(text, language='ru').replace(':', ' ').replace('_', ' ')

    #print("emoji  ", text)
    
    # Обработка HTML
    parser = MyHTMLParser()
    parser.feed(text)
    extracted_text = parser.get_data()
    # Рекурсивно применяем функцию к извлеченному тексту
    if extracted_text != text:  # Проверяем, был ли HTML тег
        text = preprocess_text(extracted_text)
    
    #print("html  ", text)
    
    # убираем знаки пунктуации
    text = remove_punctuation_list_comp(replace_math_symbols_with_words(text))

    #print("remove_punctuation_list_comp  ", text)

    text = replace_units_with_full_names(text)

    #print("replace_units_with_full_names  ", text)

    text = remove_extra_spaces_regex(remove_non_cyrillic(text))
    
    return text

In [ ]:
def lemmatize_texts(texts):
    lemmatized_texts = []
    for doc in nlp.pipe(texts):
        lemmatized_text = " ".join([token.lemma_ for token in doc])
        lemmatized_texts.append(lemmatized_text)
    return lemmatized_texts


In [ ]:
def process_csv_and_lemmatize(input_file_path, output_file_path):
    # Загрузка данных из CSV файла
    df = pd.read_csv(input_file_path)
    
    # Проверяем наличие нужного столбца
    if 'processed_comment' not in df.columns:
        raise ValueError("В файле отсутствует столбец 'processed_comment'.")

    # Пакетная лемматизация всех комментариев
    df['lemmatized_comment'] = lemmatize_texts(df['processed_comment'])

    # Сохраняем обновленный DataFrame в новый CSV файл
    df.to_csv(output_file_path, index=False)
    print("Файл успешно сохранен:", output_file_path)

In [ ]:
sample_text = " Привет :D Как дела? :) Надеюсь,  ) всё?хорошо:($100, €75, £50, f wa awfx спс ¥ 5000 и ₽3000 Ваш апи.ключ: abcd1234 ef5678gh, ваш UUID: 123e4567-e89b-12d3-a456-426614174000. +7 123 456-78-90 или (123) 456 7890. Возраст 30 лет Это пример текста с сокращениями типа кг и т.д. и HTML + <b>тегами</b>. Подробнее на www.foufos.gr  Привет 👋! Я рад этому дню 😊"
processed_text = preprocess_text(sample_text)
print(processed_text)

In [ ]:
def find_potential_contractions(file_path, column_name='comments'):
    # Регулярное выражение для поиска потенциальных сокращений
    contractions_pattern = re.compile(r'\b\w+\.\b')
    
    # Загрузка данных из файла
    df = pd.read_csv(file_path)
    
    # Убедимся, что колонка с комментариями существует
    if column_name not in df.columns:
        raise ValueError(f"Колонка {column_name} не найдена в файле.")
    
    # Поиск потенциальных сокращений в каждом комментарии
    df['potential_contractions'] = df[column_name].apply(lambda x: re.findall(contractions_pattern, x))
    
    return df[df['potential_contractions'].map(bool)]

In [ ]:
file_path = 'dataset/dataset-ru2.csv'
result_df = find_potential_contractions(file_path, 'comment')
result_df.to_csv('dataset/look2.csv', index=False)

In [ ]:
def process_csv_add_column(input_file_path, output_file_path):
    # Загрузка данных из CSV файла
    df = pd.read_csv(input_file_path)
    
    if 'comment' not in df.columns:
        raise ValueError("В файле отсутствует столбец 'comment'.")

    df['processed_comment'] = df['comment'].apply(preprocess_text)
    
    df.to_csv(output_file_path, index=False)
    print("Файл успешно сохранен:", output_file_path)

In [ ]:
input_file = 'dataset/dataset_lg.csv'  # Укажите здесь путь к вашему файлу
output_file = 'dataset/dataset_lg_train.csv'  # Путь для сохранения нового файла

process_csv_add_column(input_file, output_file)

In [ ]:
input_file_path = 'dataset_lg_train.csv'
output_file_path = 'dataset/dataset_lg_train_final.csv'
process_csv_and_lemmatize(input_file_path, output_file_path)

In [ ]:
df = pd.read_csv('dataset/train-ru2-lemmatized.csv')
print(df.shape[0])
df.head()

In [ ]:
# Загрузка данных
df = pd.read_csv('train-ru-lemmatized.csv')

# Выбор столбца (замените 'column_name' на имя вашего столбца)
column = df['lemmatized_comment']

unique_words = set()

# Подсчёт слов в каждой строке столбца
df['word_count'] = column.apply(lambda x: unique_words.update(str(x).lower().split()))

# Подсчёт общего количества слов в столбце
print(column.apply(lambda x: len(str(x))).max())

print(column.apply(lambda x: len(str(x))).mean())

print("Общее количество слов в столбце:", len(unique_words))

In [ ]:
df = pd.read_csv('train-ru-lemmatized.csv')

df['word_count'] = df['lemmatized_comment'].str.split().apply(len)

# Нахождение индекса ячейки с наибольшим количеством слов
max_words_index = df['word_count'].idxmax()

# Вывод ячейки с наибольшим количеством слов
print("Ячейка с наибольшим количеством слов:")
print(df.loc[max_words_index, 'lemmatized_comment'])

# Вывод всей строки, если нужно больше информации
print("\nСтрока с наибольшим количеством слов:")
print(df.loc[max_words_index])
print(df['comment'][250236])

In [ ]:
df = pd.read_csv('train-ru-lemmatized.csv')
df['word_count'] = df['lemmatized_comment'].str.split().apply(len)

# Вычисление среднего количества слов
average_word_count = df['word_count'].mean()

print("Среднее количество слов в ячейке:", average_word_count)

In [ ]:
df1 = pd.read_csv('dataset/dataset-ru1.csv')

# Загрузка данных из второго CSV-файла
df2 = pd.read_csv('dataset/dataset-ru2.csv')

# Объединение DataFrame
combined_df = pd.concat([df1, df2], ignore_index=True)

# Сохранение объединенного DataFrame в новый CSV-файл
combined_df.to_csv('dataset/dataset_lg.csv', index=False)

In [ ]:
def lemmatize_text(text):
    # Обработка одного текста через nlp
    doc = nlp(text)
    # Создание лемматизированной строки
    lemmatized_text = " ".join([token.lemma_ for token in doc])
    return lemmatized_text

In [ ]:
print(lemmatize_text('вот прям бесят их песни аж блевать всегда хочется клоун'))

In [ ]:
df1 = pd.read_csv('dataset/dataset-ru2.csv')
df1.head()

In [ ]:
# Загрузка файлов
df1 = pd.read_csv('dataset/dataset-ru1.csv')
df2 = pd.read_csv('dataset/dataset-ru2.csv')

# Объединение файлов
combined_df = pd.concat([df1, df2], ignore_index=True)

# Сохранение объединенного файла
combined_df.to_csv('dataset/dataset-ru.csv', index=False)

In [ ]:
df2 = pd.read_csv('dataset/dataset-ru3.csv')
df2.shape

In [ ]:
df2['comment'][239000]

In [ ]:
df2['preprocessed_comment'] = df2['comment'].apply(preprocess_text)

# Сохранение обновлённого DataFrame в новый CSV файл
df2.to_csv('updated_file-april.csv', index=False)

In [ ]:
duplicates = df2['preprocessed_comment'].duplicated(keep=False)

# Вывод строк с дублирующимися значениями
duplicate_rows = df2[duplicates]
print(duplicate_rows)

In [ ]:

ids = df2["preprocessed_comment"]
df2[ids.isin(ids[ids.duplicated()])].sort_values("preprocessed_comment")

In [ ]:
df2 = pd.read_csv('dataset/dataset_lg_train.csv')
df.dropna(inplace=True)
unique_words = set()

# Проход по каждой строке колонки и добавление слов в множество
for comment in df2['processed_comment']:
    # Делаем сплит по пробелу и добавляем результат в множество
    unique_words.update(comment.split())

# Вывод количества уникальных слов
print("Количество уникальных слов:", len(unique_words))
#print(list(unique_words)[:100])

In [ ]:
df2 = pd.read_csv('dataset/dataset_lg_train.csv')

# Удаление строк с пропущенными значениями
df2.dropna(inplace=True)

# Создание множества для уникальных слов
unique_words = set()

# Проход по каждой строке колонки и добавление слов в множество
for comment in df2['processed_comment']:
    # Делаем сплит по пробелу и добавляем результат в множество
    unique_words.update(comment.split())

# Вывод количества уникальных слов
print("Количество уникальных слов:", len(unique_words))

In [ ]:
df = pd.read_csv('dataset/dataset-ru2.csv')
df.describe()

In [ ]:
248283 + 14412